In [1]:
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
import os

spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config(
        "spark.driver.extraClassPath",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config("spark.executor.cores", '8')
    .config("spark.sql.compute.ops_on_diff_frames", True) 
    .getOrCreate())

24/04/28 20:04:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
import pyspark.pandas as ps

con="jdbc:sqlite:{}/database.sqlite".format(os.getcwd())

/home/marinevas/.asdf/installs/python/3.10.13/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

rankings = ps.read_sql("rankings", con=con)
rankings = rankings.rename(columns={'player': 'player_id'})
rankings.head(2)

,ranking_date,rank,player_id,points
0,20000110,1,101736,4135.0
1,20000110,2,102338,2915.0


In [7]:
players = ps.read_sql("players", con=con)
players.head(2)

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581


In [5]:
matches = ps.read_sql('matches', con=con)
matches = matches.rename(columns={'winner_id': 'player_id'})
# matches.dropna()
# matches['player_id'] = matches['player_id'].astype(int)
matches.head(2)

24/04/28 20:04:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,player_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner1_id,winner2_id,loser1_id,loser2_id,winner1_name,winner1_hand,winner1_ht,winner1_ioc,winner1_age,winner2_name,winner2_hand,winner2_ht,winner2_ioc,winner2_age,loser1_name,loser1_hand,loser1_ht,loser1_ioc,loser1_age,loser2_name,loser2_hand,loser2_ht,loser2_ioc,loser2_age,winner1_rank,winner1_rank_points,winner2_rank,winner2_rank_points,loser1_rank,loser1_rank_points,loser2_rank,loser2_rank_points
0,2017-M020,Brisbane,Hard,32,A,20170102.0,271,104678.0,None,None,Viktor Troicki,R,193.0,SRB,30.8,106415.0,None,Q,Yoshihito Nishioka,L,170.0,JPN,21.2,6-4 7-5,3,R32,91.0,11.0,5.0,64.0,45.0,35.0,6.0,11.0,1.0,3.0,0.0,1.0,82.0,53.0,33.0,13.0,11.0,6.0,10.0,29.0,1385.0,100.0,604.0,NaN,None,None,NaN,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-M020,Brisbane,Hard,32,A,20170102.0,272,106378.0,None,None,Kyle Edmund,R,188.0,GBR,21.9,124014.0,None,Q,Ernesto Escobedo,R,185.0,USA,20.4,7-6(4) 7-6(6),3,R32,130.0,11.0,2.0,83.0,48.0,37.0,19.0,12.0,2.0,3.0,11.0,3.0,113.0,67.0,39.0,27.0,12.0,9.0,10.0,45.0,1001.0,141.0,443.0,NaN,None,None,NaN,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


count the number of matches and average rank of players with rankings above 100

o tempo deu sempre o mesmo para 2, 4 e 8 cores...

In [4]:
query = '''
SELECT players.name_first, players.name_last, 
       AVG(rankings.rank) AS avg_rank, 
       COUNT(matches.match_num) AS total_matches
FROM players
JOIN rankings ON players.player_id = rankings.player
JOIN matches ON players.player_id = matches.winner_id
WHERE rankings.rank <= 100
GROUP BY players.name_first, players.name_last
ORDER BY avg_rank DESC
'''

In [5]:
df = ps.read_sql_query(query, con=con)
df.head(2)

,name_first,name_last,avg_rank,total_matches
0,Alex,Lopez Moron,100.000000000000000000,536.000000000000000000
1,Francisco,Montana,100.000000000000000000,91.000000000000000000


In [8]:
ps.set_option('compute.ops_on_diff_frames', True)

df_result = rankings.join(players.set_index('player_id'), on='player_id') \
                    .join(matches.set_index('player_id'), on='player_id') 
                    # .filter(rankings.rank <= 100) \
                    # .groupBy("name_first", "name_last") \
                    # .agg({"rank": "avg", "match_num": "count"}) \
                    # .orderBy("avg(rank)", ascending=False)

df_result.head(3)
# df_result = players.join(rankings, players.player_id == rankings.player) \
#                      .join(matches, players.player_id == matches.winner_id) \
#                      .filter(rankings.rank <= 100) \
#                      .groupBy("name_first", "name_last") \
#                      .agg({"rank": "avg", "match_num": "count"}) \
#                      .orderBy("avg(rank)", ascending=False)



24/04/28 20:05:35 WARN MemoryStore: Not enough space to cache rdd_38_0 in memory! (computed 130.2 MiB so far)
24/04/28 20:05:35 WARN BlockManager: Persisting block rdd_38_0 to disk instead.
24/04/28 20:08:28 WARN BlockManager: Putting block rdd_38_0 failed due to exception java.io.IOException: No space left on device.
24/04/28 20:08:28 WARN BlockManager: Block rdd_38_0 could not be removed as it was not found on disk or in memory
24/04/28 20:08:28 ERROR Executor: Exception in task 0.0 in stage 13.0 (TID 8)
java.io.IOException: No space left on device
	at java.base/sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at java.base/sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:62)
	at java.base/sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:113)
	at java.base/sun.nio.ch.IOUtil.write(IOUtil.java:79)
	at java.base/sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:280)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:354)
	at java.base/java.ni

Py4JJavaError: An error occurred while calling o2329.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 8) (mari executor driver): java.io.IOException: No space left on device
	at java.base/sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at java.base/sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:62)
	at java.base/sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:113)
	at java.base/sun.nio.ch.IOUtil.write(IOUtil.java:79)
	at java.base/sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:280)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:354)
	at java.base/java.nio.channels.Channels.writeFullyImpl(Channels.java:74)
	at java.base/java.nio.channels.Channels.writeFully(Channels.java:97)
	at java.base/java.nio.channels.Channels$1.write(Channels.java:172)
	at org.apache.spark.storage.memory.RedirectableOutputStream.wri

24/04/28 20:08:57 WARN MemoryStore: Not enough space to cache rdd_60_0 in memory! (computed 130.2 MiB so far)
24/04/28 20:08:57 WARN BlockManager: Persisting block rdd_60_0 to disk instead.
24/04/28 20:11:16 WARN BlockManager: Putting block rdd_60_0 failed due to exception java.io.IOException: No space left on device.
24/04/28 20:11:16 WARN BlockManager: Block rdd_60_0 could not be removed as it was not found on disk or in memory
24/04/28 20:11:16 ERROR Executor: Exception in task 0.0 in stage 24.0 (TID 15)
java.io.IOException: No space left on device
	at java.base/sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at java.base/sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:62)
	at java.base/sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:113)
	at java.base/sun.nio.ch.IOUtil.write(IOUtil.java:79)
	at java.base/sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:280)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:354)
	at java.base/java.n

Py4JJavaError: An error occurred while calling o2445.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 15) (mari executor driver): java.io.IOException: No space left on device
	at java.base/sun.nio.ch.FileDispatcherImpl.write0(Native Method)
	at java.base/sun.nio.ch.FileDispatcherImpl.write(FileDispatcherImpl.java:62)
	at java.base/sun.nio.ch.IOUtil.writeFromNativeBuffer(IOUtil.java:113)
	at java.base/sun.nio.ch.IOUtil.write(IOUtil.java:79)
	at java.base/sun.nio.ch.FileChannelImpl.write(FileChannelImpl.java:280)
	at org.apache.spark.storage.CountingWritableChannel.write(DiskStore.scala:354)
	at java.base/java.nio.channels.Channels.writeFullyImpl(Channels.java:74)
	at java.base/java.nio.channels.Channels.writeFully(Channels.java:97)
	at java.base/java.nio.channels.Channels$1.write(Channels.java:172)
	at org.apache.spark.storage.memory.RedirectableOutputStream.wr